<a href="https://colab.research.google.com/github/elifmacit/booksearch/blob/main/booksearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# ---- Veritabanına bağlan ----
conn = sqlite3.connect("books.db")

# ---- SQL ile aday kitapları çek (serbest sorgu) ----
def get_candidates(query):
    like = f"%{query.lower()}%"
    sql = """
    SELECT
        rowid AS id,
        COALESCE(title,'') AS title,
        COALESCE(authors,'') AS authors,
        COALESCE(publisher,'') AS publisher,
        COALESCE(average_rating,0) AS average_rating
    FROM books
    WHERE lower(title)    LIKE ?
       OR lower(authors)  LIKE ?
       OR lower(publisher)LIKE ?
    LIMIT 200;
    """
    return pd.read_sql(sql, conn, params=[like, like, like])

# ---- Arama fonksiyonu ----
def search_books(query, top_k= 20, alpha=0.5):
    df = get_candidates(query)

    if len(df) == 0:
        print(f"❗ '{query}' için veri bulunamadı.")
        return

    # Metin alanını birleştir
    df["text"] = (
        df["title"].str.lower() + " " +
        df["authors"].str.lower() + " " +
        df["publisher"].str.lower()
    )

    # TF-IDF vektörleri oluştur
    X = vectorizer.fit_transform(df["text"])
    q_vec = vectorizer.transform([query.lower()])

    # Kosinüs benzerliği
    cosine_scores = cosine_similarity(q_vec, X).ravel()

    # Rating'i normalize et (0-1)
    r = df["average_rating"].astype(float).values
    r_norm = (r - r.min()) / (r.max() - r.min() + 1e-9)

    # Ağırlıklı skor = cosine + rating
    final_score = (1 - alpha) * cosine_scores + alpha * r_norm

    # Sırala
    idx = np.argsort(-final_score)[:top_k]
    results = df.iloc[idx]

    # Sonuç yoksa
    if cosine_scores.max() < 0.01:
        print(f"❗ '{query}' ile ilgili anlamlı sonuç bulunamadı.")
        return

    # Yazdır
    print(f"\n🔎 Arama: '{query}' — Toplam {len(results)} sonuç\n")
    for _, row in results.iterrows():
        print(f"- {row['title']} — {row['authors']} (Rating: {row['average_rating']})")

# ---- Kullanıcı etkileşimi ----
print("📚 Kitap Arama Sistemi — boş Enter = çıkış")
while True:
    q = input("\nSorgu: ")
    if q.strip() == "":
        print("👋 Çıkış yapıldı.")
        break
    search_books(q)


📚 Kitap Arama Sistemi — boş Enter = çıkış
